In [14]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
import pandas as pd
from sklearn.cluster import AffinityPropagation
from sklearn import cluster
import nltk
from nltk.corpus import stopwords
import re
from difflib import SequenceMatcher
from collections import Counter

In [27]:
# called in data_cleaning
def cleansing_special_characters(txt):
    seps = [' ',';',':','.',',','*','#','@','|','\\','-','_','?','%','!','^','(',')','$','=','+','"','<','>',"'",]
    default_sep = seps[0]
    
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    re.sub(' +', ' ', txt)
    temp_list = [i.strip() for i in txt.split(default_sep)]
    temp_list = [i for i in temp_list if i]
    return " ".join(temp_list)

# =================================================================================================

# called in data_cleaning
def clean_stopword(txt):
    temp_list = txt.split(" ")
    temp_list = [i for i in temp_list if i not in stopwords.words('english')]
    return " ".join(temp_list)

# ==================================================================================

In [28]:
# called in company_clusters
def data_cleaning(data,nameCol , dropForeign =True):
    data.dropna(subset=[nameCol], inplace =True)
    data = data.rename_axis("CompanyID").reset_index()
    data['nonAscii_count'] = data[nameCol].apply(lambda x : sum([not c.isascii() for c in x]))
    if dropForeign:
        data = data[data.nonAscii_count ==0]
    else:
        pass
    data.drop('nonAscii_count', axis =1, inplace= True)
    data_clean = data.copy()
    data_clean['CustomerName_clean'] = data_clean[nameCol].apply(lambda x: x.lower())
    data_clean['CustomerName_clean'] = data_clean['CustomerName_clean'].apply(cleansing_special_characters)
    data_clean['CustomerName_clean'] = data_clean['CustomerName_clean'].apply(clean_stopword)
    return data_clean

# ===========================================================================================

# called in company_clusters
def fuzz_similarity(cust_names):
    similarity_array = np.ones((len(cust_names),(len(cust_names))))*100
    
    for i in range(1,len(cust_names)):
        for j in range(i):
            s1 = fuzz.token_set_ratio(cust_names[i],cust_names[j]) + 0.000000000001
            s2 = fuzz.partial_ratio(cust_names[i],cust_names[j]) + 0.00000000001
            similarity_array[i][j] = 2*s1*s2/(s1+s2)
#             print(similarity_array[i][j])
    
    for i in range(len(cust_names)):
        for j in range(i+1, len(cust_names)):
            similarity_array[i][j] = similarity_array[j][i]
    
    np.fill_diagonal(similarity_array,100)
    return similarity_array

# ========================================================================

#output to standard_name
def company_clusters(data, nameCol, dropForeign = True):
    data_clean = data_cleaning(data, nameCol=nameCol, dropForeign=dropForeign)
    cust_names = data_clean.CustomerName_clean.to_list()
    cust_ids = data_clean.CompanyID.to_list()
    
    similarity_array = fuzz_similarity(cust_names)
    clusters = cluster.AffinityPropagation(affinity = 'precomputed').fit_predict(similarity_array)
    df_cluster = pd.DataFrame(list(zip(cust_ids, clusters)), columns=['CompanyID','cluster'])
    
    df_eval = df_cluster.merge(data_clean, on = 'CompanyID',how='left')
    return df_eval


def standard_name(df_eval):
    d_standard_name = {}
    for cluster in df_eval.cluster.unique():
        names = df_eval[df_eval['cluster']==cluster].CustomerName_clean.to_list()
        l_common_substring = []
        if len(names)>1:
            for i in range(0, len(names)):
                for j in range(i+1, len(names)):
                    seqMatch = SequenceMatcher(None, names[i],names[j])
                    match = seqMatch.find_longest_match(0, len(names[i]), 0, len(names[j]))
                    if (match.size!= 0):
                        l_common_substring.append(names[i][match.a :match.a + match.size].strip())
            n = len(l_common_substring)
            counts = Counter(l_common_substring)
            get_mode = dict(counts)
            mode = [k for k,v in get_mode.items() if v == max(list(counts.values()))]
            d_standard_name[cluster] = ";".join(mode)
        else:
            d_standard_name[cluster] = names[0]
            
    df_standard_names = pd.DataFrame(list(d_standard_name.items()), columns=['cluster', 'StandardName'])
    df_eval = df_eval.merge(df_standard_names, on = 'cluster', how = 'left')
    df_eval['Score_with_standard'] = df_eval.apply(lambda x: fuzz.token_set_ratio(x['StandardName'],x['CustomerName_clean']),axis =1)
    df_eval['standard_name_withoutSpaces'] = df_eval.StandardName.apply(lambda x: x.replace(" ",""))
    
    for name in df_eval.standard_name_withoutSpaces.unique():
        if len(df_eval[df_eval.standard_name_withoutSpaces==name].cluster.unique()) > 1:
            df_eval.loc[df_eval.standard_name_withoutSpaces==name, 'StandardName'] = name
            
    return df_eval.drop('standard_name_withoutSpaces', axis=1)

In [29]:
supp = pd.read_excel('2000_Aug_Imp-20.xlsx')

In [31]:
column_name ='BUYER_NAME'
final = standard_name(company_clusters(supp,column_name))

C:\Users\Manish\anaconda3\lib\site-packages\sklearn\cluster\_affinity_propagation.py:226: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  "will not have any cluster centers.", ConvergenceWarning)


In [32]:
final.to_excel('test.xlsx')